In [5]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

## 福尔摩斯推理

In [6]:
model = BayesianNetwork([('B', 'A'), ('E', 'A'), ('A', 'W'), ('E', 'R')])
cpd_1 = TabularCPD(variable='B', variable_card=2, values=[[0.9], [0.1]])
cpd_2 = TabularCPD(variable='E', variable_card=2, values=[[0.99], [0.01]])

In [9]:
cpd_A = TabularCPD(variable='A', variable_card=2,
                   values=[[0.99, 0.1, 0.1, 0.01],
                           [0.01, 0.9, 0.9, 0.99]],
                   evidence=['E', 'B'],#父节点
                   evidence_card=[2, 2])#警铃
cpd_W = TabularCPD(variable='W', variable_card=2,
                   values=[[0.99, 0.35],
                           [0.01, 0.65]],
                   evidence=['A'],#父节点
                   evidence_card=[2])#华生致电福尔摩斯
cpd_R = TabularCPD(variable='R', variable_card=2,
                     values=[[0.999, 0.01],
                            [0.001, 0.99]],
                     evidence=['E'],#父节点
                     evidence_card=[2])#广播播报地震

### 问题1

警铃A响了，那么窃贼入室B的概率是多少？

In [12]:
model.add_cpds(cpd_1, cpd_2, cpd_A, cpd_W, cpd_R)
infer = VariableElimination(model)
print(infer.query(variables=['B'],evidence={'A':1}))

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.1588 |
+------+----------+
| B(1) |   0.8412 |
+------+----------+


### 问题2

福尔摩斯迅速开车回家。在路上，他听广播家那里发生了地震。请问他应该回家抓贼还是迅速撤离该地区以躲避地震？

比较A=1,R=1时B和E谁的概率更大

In [13]:
print(infer.query(variables=['E'],evidence={'A':1,'R':1}))

+------+----------+
| E    |   phi(E) |
+======+==========+
| E(0) |   0.0108 |
+------+----------+
| E(1) |   0.9892 |
+------+----------+


In [14]:
print(infer.query(variables=['B'],evidence={'A':1,'R':1}))

+------+----------+
| B    |   phi(B) |
+======+==========+
| B(0) |   0.8825 |
+------+----------+
| B(1) |   0.1175 |
+------+----------+


## 草地喷水器推理

In [16]:
model = BayesianNetwork([('R', 'S'), ('R', 'G'), ('S', 'G')])
cpd_1 = TabularCPD(variable='R', variable_card=2, values=[[0.8], [0.2]])
cpd_S = TabularCPD(variable='S', variable_card=2,
                   values=[[0.6, 0.99],
                           [0.4, 0.01]],
                   evidence=['R'],#父节点
                   evidence_card=[2])
cpd_G = TabularCPD(variable='G', variable_card=2,
                   values=[[1.0, 0.2, 0.1, 0.01],
                           [0.0, 0.8, 0.9, 0.99]],
                   evidence=['S', 'R'],#父节点
                   evidence_card=[2, 2])#警铃

### 问题一：

计算草地是湿的情况下，下雨的概率？

In [17]:
model.add_cpds(cpd_1, cpd_S, cpd_G)
infer = VariableElimination(model)
print(infer.query(variables=['R'],evidence={'G':1}))

+------+----------+
| R    |   phi(R) |
+======+==========+
| R(0) |   0.6423 |
+------+----------+
| R(1) |   0.3577 |
+------+----------+


### 问题二：

草地是湿的情况下，自动喷水的概率？

In [18]:
print(infer.query(variables=['S'],evidence={'G':1}))

+------+----------+
| S    |   phi(S) |
+======+==========+
| S(0) |   0.3533 |
+------+----------+
| S(1) |   0.6467 |
+------+----------+
